In [0]:
%run "/Databricks_MasterClass/Tutorial"

# DELTA LAKE

In [0]:
df_sales.write.format('delta') \
    .mode('overwrite') \
     .option('path', 'abfss://destination@datalakemclassstorage.dfs.core.windows.net/Sales') \
    .save()
      # .save('abfss://destination@datalakemclassstorage.dfs.core.windows.net/Sales')

## Managed vs External Delta Tables"

In [0]:
df_sales.createOrReplaceTempView('salestemp')

In [0]:
%sql
select * from salestemp

### Create Database

In [0]:
%sql
use catalog hive_metastore

In [0]:
%sql
create database if not exists salesdb

### Create Managed Table

In [0]:
%sql
create or replace table salesdb.mansales 
using delta
as
select * from read_files('abfss://source@datalakemclassstorage.dfs.core.windows.net/Sales.csv', 
format => 'csv', header => 'true', inferSchema => 'true' )

In [0]:
%sql
select * from salesdb.mansales limit 5

In [0]:
%sql
--describe extended salesdb.mansales
drop table salesdb.mansales

### Create External Table

In [0]:
%sql
drop table hive_metastore.salesdb.extsales

In [0]:
%sql
create table hive_metastore.salesdb.extsales
using delta
location 'abfss://destination@datalakemclassstorage.dfs.core.windows.net/salesdb/extsales'
as select * from salestemp

In [0]:
%sql
drop table hive_metastore.salesdb.extsales


In [0]:
%sql
create table hive_metastore.salesdb.exttable
(
  id int,
  name string,
  age int
)
using delta
location 'abfss://source@datalakemclassstorage.dfs.core.windows.net/salesdb/exttable'

In [0]:
%sql
insert into table hive_metastore.salesdb.exttable values (1, 'jane', 40), (2, 'john', 30), (3, 'jack', 20)

In [0]:
%sql
delete from hive_metastore.salesdb.exttable where id = 3

In [0]:
%sql
insert into table hive_metastore.salesdb.exttable values (4, 'jill', 50), (5, 'joe', 60), (6, 'jake', 70)

In [0]:
%sql
describe history hive_metastore.salesdb.exttable

In [0]:
%sql
select distinct Outlet_Establishment_Year
 from hive_metastore.salesdb.extsales limit 10

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,


Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales

In [0]:
%sql
select Outlet_size,outlet_type, count(*), avg(Item_Outlet_Sales) from hive_metastore.salesdb.extsales group by outlet_type,Outlet_size --having Outlet_Size is null

In [0]:
%sql
update hive_metastore.salesdb.extsales 
set Item_weight = 0 where Item_weight is null
    


In [0]:
%sql
update hive_metastore.salesdb.extsales
set Outlet_size = Case when (Item_outlet_sales between 1000 and 2000) then 'Medium' when (Item_outlet_sales > 2000) then 'High' else 'Small' end where Outlet_size is null

In [0]:
%sql
select * from hive_metastore.salesdb.extsales --where item_identifier = 'FDH17'

In [0]:
%sql
optimize hive_metastore.salesdb.extsales zorder by (item_identifier)

In [0]:
%sql
describe history hive_metastore.salesdb.extsales

In [0]:
%sql
alter table hive_metastore.salesdb.extsales
add columns (Identifier_id int)

In [0]:
%sql
update hive_metastore.salesdb.extsales
set Identifier_id = substring(Outlet_identifier,4)